<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
# Данная библиотека импортируется ИСКЛЮЧИТЕЛЬНО для возможности отображения графиков через
# https://nbviewer.org/ Jupyter Notebook. Для действий над данными, НЕ ИСПОЛЬЗУЕТСЯ!!!
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

import plotly.express as px
import plotly.graph_objects as go

import sqlalchemy
from sqlalchemy.sql import text
import pandas as pd

import requests
from bs4 import BeautifulSoup

# Данная библиотека импортируется ИСКЛЮЧИТЕЛЬНО для возможности отображения графиков через
# https://nbviewer.org/ Jupyter Notebook. Для действий над данными, НЕ ИСПОЛЬЗУЕТСЯ!!!
import plotly.io as pio


In [2]:
DBNAME = ''
USER = ''
PASSWORD = ''
HOST = ''
PORT = 

In [3]:
url = f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}'
engine = sqlalchemy.create_engine(url)

# Юнит 3. Предварительный анализ данных

Небольшая оптимизация кода, а именно написание дополнительных функций с целью добиться уменьшения кода и удобство в его читании

In [4]:
# Учитывая, что часто будет выполнятся такое действие, 
# как выполнение запроса, напишем функцию для этого 

def sqlExecution(sql):
  """_summary_
  Выполняем SQL запрос
  Args:
      sql (sql): Строковая переменная содержащая SQL запрос

  Returns:
      Pandas DataFrame: Возвращает результат выполнения запроса
  """
  with engine.connect().execution_options(autocommit=True) as conn:
    return pd.read_sql(sql, con = conn)

# Учитывая что все 4 последующих запроса для юнита 3 структурно идентичны, 
# и будет менятся только таблица из которой будет происходить выборка
# чтобы не писать каждый раз новый запрос, напишем функцию, 
# где входным параметром будет название таблицы, и возвращать функция будет количество записей 

def countIdRecords(table) -> int:
  """_summary_
  Функция возвращает количество записей в таблице

  Args:
      table (string): название таблицы

  Returns:
      int: количество записей
  """
  sql = f'''
    SELECT 
      count(*) as {table}_count 
    FROM public.{table};
  '''
  return sqlExecution(sql).iloc[0,0]

***
1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# результат запроса
# Для таблицы Vacancies количество записей равно :
countIdRecords('vacancies')

49197

***
2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# результат запроса
# Для таблицы Employers количество записей равно :
display(countIdRecords('employers'))

23501

***
3. Посчитайте с помощью запроса количество регионов (таблица areas). 

In [7]:
# результат запроса
# Для таблицы Areas количество записей равно :
display(countIdRecords('areas'))

1362

***
4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица industries).

In [8]:
# результат запроса
# Для таблицы Industries количество записей равно :
display(countIdRecords('industries'))

294

***

### Выводы для предварительного анализа

Первоначальный взгляд на таблицы базы данных project_sql показал нам, что мы имеем дело с:
1. около 50к вакансий
2. 23.5к работодателей
3. Работа может предлагаться в 1362 городах, столько городов имеется в базе
4. И работа может быть представлена в 294 различных сферах

Без сравнения с другими базами вакансий тяжело сказать, относится ли данная база HH.RU к большим базам или нет. Одно можно сказать точно, она не маленькая.
***


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [9]:
sql_u4_1 = f'''
    SELECT 
      a.name as area
      ,count(*) as cnt 
    FROM public.areas as a
    INNER JOIN public.vacancies as v on a.id = v.area_id
    GROUP by a.name
    ORDER by cnt desc
    LIMIT 5
    ;
  '''


In [10]:
display(sqlExecution(sql_u4_1))

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


***
2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [11]:
sql_u4_2 = f'''
    SELECT 
        count(*) as cnt
    FROM public.vacancies as v
    WHERE v.salary_from is not null 
        or v.salary_to is not null  
    ;
  '''

In [12]:
display(sqlExecution(sql_u4_2))

,cnt
0,24073


***
3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [13]:
sql_u4_3 = f'''
    SELECT 
        avg(salary_from) as avg_salary_from
        ,avg(salary_to) as avg_salary_to
    FROM public.vacancies as v
    ;
  '''

In [14]:
display(sqlExecution(sql_u4_3).round())

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


***
4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [15]:
sql_u4_4 = f'''
    SELECT 
        v.schedule
        ,v.employment
        ,count(*) as vacancies_count
    FROM public.vacancies as v
    GROUP by v.schedule, v.employment
    ORDER by vacancies_count desc
    ;
  '''

In [16]:
display(sqlExecution(sql_u4_4))

,schedule,employment,vacancies_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


***
5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [17]:
sql_u4_5 = f'''
    SELECT 
        v.experience
        ,count(*) as vacancies_count
    FROM public.vacancies as v
    GROUP by v.experience
    ORDER by vacancies_count asc
    ;
  '''

In [18]:
display(sqlExecution(sql_u4_5))

,experience,vacancies_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

### Выводы для детального анализа вакансий

Наибольшее количество предлагаемых ваканский относится к крупным городам.
1. Наибольшее количество предложений по работе в Москве - 5333 вакансии, далее следуют Санкт-Петербург, Минск, Новосибирск, Алматы с 2851, 2112, 2006 и 1892 вакансий соотвественно.
Чем больше город, тем больше предложений по работе. 
Так же можем отметить, что предложения по работам не только на территории России, но также и других государств.
2. Средняя предлагаемая зарплата находится в диапазоне 71065 - 110537 Российских рублей (при учитывании только вакансий с указанными данными по зарплате, так как только у 24073 вакансий из 50К указано как минимум одно из полей мин. или макс. предлагаемой зарплаты)
3. Наибольшее количество предложений по работе предполагают Полную занятость и Полный рабочий день - 35367, на втором месте с большим отрывом идет Полный рабочий день и Удаленная работа - 7802 вакансии
4. Наименее редкое предложение для потенциальных кндидатов иметь опыт работы более 6 лет - 1337 вакансий, наибольшее от 1 года до  лет, и составляет 26152 вакансии. Также для людей совсем без опыта приходится 7197 вакансий.

***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [19]:
sql_u5_1 = f'''
    (SELECT 
        e.name
        ,count(*) as vacancies_count
    FROM public.vacancies as v
    INNER JOIN public.employers as e on e.id = v.employer_id
    GROUP by e.name
    ORDER by vacancies_count desc
    LIMIT 1)
    
    UNION
    
    (SELECT 
        e.name
        ,count(*) as vacancies_count
    FROM public.vacancies as v
    INNER JOIN public.employers as e on e.id = v.employer_id
    GROUP by e.name
    ORDER by vacancies_count desc
    LIMIT 1
    OFFSET 4)
    ORDER BY name DESC
    ;
  '''

In [20]:
display(sqlExecution(sql_u5_1))

,name,vacancies_count
0,Яндекс,1933
1,Газпром нефть,331


***
2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [21]:
sql_u5_2 = f'''
    SELECT
        a.name 
        ,count(distinct e.id) as employers_count
        ,count(distinct v.id) as vacancies_count
    FROM public.areas as a
    LEFT JOIN public.employers as e on e.area = a.id
    LEFT JOIN public.vacancies as v on v.area_id = a.id
    WHERE v.id is null
    GROUP by a.name
    ORDER by employers_count desc
    LIMIT 1
    ;
  '''

In [22]:
display(sqlExecution(sql_u5_2))

,name,employers_count,vacancies_count
0,Россия,410,0


***
3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [23]:
sql_u5_3 = f'''
    SELECT
        e.name 
        ,count(distinct v.area_id) as areas_count
    FROM public.employers as e
    LEFT JOIN public.vacancies as v on v.employer_id = e.id
    GROUP by e.name
    ORDER by areas_count desc
    ;
  '''

In [24]:
display(sqlExecution(sql_u5_3))

,name,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


***
4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [25]:
sql_u5_4 = f'''
    SELECT
      count(e.id) as cnt
    FROM public.employers as e
    LEFT JOIN public.employers_industries as ei on ei.employer_id = e.id
    WHERE ei.employer_id is null
    ;
  '''

In [26]:
display(sqlExecution(sql_u5_4))

,cnt
0,8419


***
5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [27]:
sql_u5_5 = f'''
    SELECT
      e.name
      ,count(distinct ei.industry_id) as cnt
    FROM public.employers as e
    LEFT JOIN public.employers_industries as ei on ei.employer_id = e.id
    WHERE ei.employer_id is not null
    GROUP by e.name
    HAVING count(distinct ei.industry_id) = 4
    ORDER by e.name asc
    LIMIT 1
    OFFSET 2
    ;
  '''

In [28]:
display(sqlExecution(sql_u5_5))

,name,cnt
0,2ГИС,4


***
6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [29]:
sql_u5_6 = f'''
    SELECT
      count(e.name) as cnt
    FROM public.employers as e
    LEFT JOIN public.employers_industries as ei on ei.employer_id = e.id
    INNER JOIN public.industries as i on i.id = ei.industry_id
    WHERE i.name = 'Разработка программного обеспечения'
    ;
  '''

In [30]:
display(sqlExecution(sql_u5_6))

,cnt
0,3553


***
7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [31]:
# используем метод pandas read_html, который позволяет считывать все элементы с тэгом table
# с указанного url в DataFrame. К сожалению у текущей статьи нету отдельного идентификаторв таблицы, 
# поэтому получаем все таблицы со страницы. необходимая нам лежит под индексом 1
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
cities = list(pd.read_html(url)[1]['Город'])
# Один из городов указан в виде Воронеж[а]. Удалим ненужный аттрибут [а] в конце
cities[13] = cities[13][:-3]


In [32]:
# обьявлю отдельную переменную для компании, так явнее видно для какой компании происходит выборка
company = 'Яндекс'

# Также у нас есть повторяющийся код SQL, обернем его тоже в отдельную переменную 
# И дополнительно получим список городов для условия город из списка, путем применения join для списка городов
common_sql = f'''FROM public.employers as e
    LEFT JOIN public.vacancies as v on v.employer_id = e.id
    INNER JOIN public.areas as a on v.area_id = a.id 
    WHERE 
          e.name = '{company}'
      and v.id is not null
      and a.name in ({','.join(f'{chr(0x27)}{c}{chr(0x27)}' for c in cities)})
    '''
# окончательный запрос
sql_u5_7 = f'''
    (SELECT
      a.name
      ,count(v.id) as cnt
    {common_sql}
    GROUP BY 1
    ORDER BY 2 asc)
    
    union
    
    (SELECT
      'Total'
      ,count(v.id) as cnt
        {common_sql}
    )
    ORDER BY 2
    ;
  '''

In [33]:
display(sqlExecution(sql_u5_7))

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

### Выводы по анализу работодателей

В базе присутствуют, как крупные работадатели, которые представлены во многих городах/странах, так и небольшие, у которых на данный момент нету вакансий от слова вообще.

Бросается в глаза, что больше всего вакансий приходится на район Россия, что несколько странно, это или говорит о какой-то специфике вакансии, возможно это удаленная работа и имеет только смысл страна, с точки зрения налогового законадательства или недостаточно/некорректно указанная информация о городе. Этот момент надо детальнее проверить.

Интересен также момент, что компания с названием Поляков Денис Иванович предлагает работу почти в 90 регионах, потому как это очень похоже на ИП, а размер ИП вряд ли позвоялет содержать такое количество сотрудников под ИП. Возможно выброс, надо изучать детальнее.

Некоторые работодатели не указывают сферы в которых они рабоают, это не критично, но думаю было бы хорошо, было досабрать эти даные и в будущем для них можно было бы отдельно предлагать специалистов, сотрудничая с их HR отделами, в случае если это больше компании. В случе мелких нет смысла и хватает треболваний указанных в вакансии.



# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [34]:
sql_u6_1 = f'''
    SELECT
      count(distinct v.id) as cnt
    FROM public.vacancies as v
    -- выбираем записи в любом регистре где в признаке name (название вакансии)
    -- присутствет запись или 'data' или 'данн' 
    WHERE v.name ~~* any (array[
      '%%data%%'
      ,'%%данн%%'
    ])
    ;
  '''

In [35]:
display(sqlExecution(sql_u6_1))

,cnt
0,1771


***
2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [36]:
# Дополнительный критерий, который выбирает только Junior вакансии
juniors ='''   
     (
      v.employment = 'Стажировка' 
      or v.experience = 'Нет опыта' 
      or v.name ~~* '%%junior%%'
      ) 
      '''
# Беря во внимание, что последующие запросы будут выбирать только вакансии связанные с DS, чтобы не писать каждый раз это в условии и не ошибится, 
# или в случае если нам придется расширить список или уменьшить, введем переменную where, в которой будет идти отбор только DS вакансий   
where_DS_Only = '''    
      (
        v.name ~~* any (array[
          '%%data%%scientist%%',
          '%%data%%science%%',
          '%%исследователь%%данных%%',
          '%%machine%%learning%%',
          '%%машинн%%обучен%%'      
        ])
      or (
          v.name !~~* '%%HTML%%'
          and v.name ~~ '%%ML%%'
        )
      )
'''
sql_u6_2 = f'''
    SELECT
      count(*) as cnt
    FROM public.vacancies as v
    WHERE
    --общее условие, только DS вакансии
    {where_DS_Only}
    and
    --Дополнительное условие, ограничивающее только до Junior
    {juniors}
    ;
  '''

In [37]:
sqlExecution(sql_u6_2)

,cnt
0,51


***
3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [38]:
# используем структуру запроса WITH, для более удобного чтения и понимания.
# Из уже выбранного множества только DS вакансии, выберем только тех, у кого имеется один из ключевых навыков SQL или postgreSQL
sql_u6_3 = f'''
    WITH DSOnly as (
    SELECT
      *
    FROM public.vacancies as v
    WHERE
    {where_DS_Only}
    )
    
    SELECT count(*) as cnt
    FROM DSOnly ds
    -- используем case insensitive подход при поиске
    WHERE ds.key_skills ~~* any (array[
      '%%sql%%',
      '%%postgres%%'
    ])  
      
    ;
  '''

In [39]:
sqlExecution(sql_u6_3)

,cnt
0,201


***
4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [40]:
# используем структуру запроса WITH, для более удобного чтения и понимания.
# Из уже выбранного множества только DS вакансии, выберем только тех, у кого имеется один из ключевых навыков python
sql_u6_4 = f'''
    WITH DSOnly AS (
    SELECT
      *
    FROM public.vacancies as v
    WHERE
    {where_DS_Only}
    )
    
    SELECT count(*) as cnt
    FROM DSOnly ds
    -- используем case insensitive подход при поиске
    WHERE ds.key_skills ~~* any (array[
      '%%python%%'
    ])  
      
    ;
  '''

In [41]:
sqlExecution(sql_u6_4)

,cnt
0,351


***
5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [42]:
# используем структуру запроса WITH, для более удобного чтения и понимания.
# Из уже выбранного множества только DS вакансии,высчитываем среднее количество навыков
sql_u6_5 = f'''
    with DSOnly as (
    SELECT
      *
    FROM public.vacancies as v
    WHERE
    {where_DS_Only}
    )
    
    select 
      --округляем до 2 знаков после запятой
      ROUND(
        --высчитываем среднее по всем записям
        AVG(
          -- chr(9) ASCII код \t. Признак key_skills использует это для разделения навыков 
          -- Поэтому для вычисления кол-ва навыков прибегаем к простому способу, удаляем через replace \t 
          -- прибавляем 1-цу, так как разделитель в конце не ставится и поэтому навыков на 1 больше, чем разделителей
          -- и полученный число отнимаем от всей длины строки, тем самым понимая сколько навыков указано
          LENGTH(ds.key_skills) - LENGTH(replace(ds.key_skills, CHR(9), '')) +1
        ),2
      ) 
    from DSOnly ds
    ;
  '''

In [43]:
sqlExecution(sql_u6_5)

,round
0,6.41


***
6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [44]:
# используем структуру запроса WITH, для более удобного чтения и понимания.
# Из уже выбранного множества только DS вакансии,высчитываем среднее количество навыков
sql_u6_6 = f'''
    WITH DSOnly AS (
    SELECT
      *
    FROM public.vacancies as v
    WHERE
    {where_DS_Only}
    )
    
    SELECT
      --округляем до 2 знаков после запятой
      ROUND(
          --высчитываем среднее по всем записям
          AVG(
            -- так как по условию задачи необходимо посчитать среднюю зарплату, при условии, что зарплата указана 
            -- как и минимальная так и максимальная для записи, или же в случае если уцказана только одна из эарплат, взять ее
            -- применяем функцию coalesce, где входными параметрами являются приоритеты расчета зарплат указанные по их приоритету
            coalesce((ds.salary_from+ds.salary_to)/2, ds.salary_from, ds.salary_to) 
        ),0) as average_salary_DS
    FROM DSOnly ds
    WHERE 
     (
          -- уменьшим обьем выборки, хотя этот шаг необязателен. Функции агрегации опускают Null поля
          -- Тут больше вопрос быстродействия, а это надо посчитать.
          ds.salary_from is not null 
       or ds.salary_to is not null
      )
      and
      ds.experience = 'От 3 до 6 лет'
    ;
  '''

In [45]:
sqlExecution(sql_u6_6)

,average_salary_ds
0,243115.0


***

### Выводы по предметному анализу 

10.5% вакансий DS специалистов относятся к Junior-ам, что показывает, что людям без опыта в данной области тяжелее всего найти работу.

Также часто требуемыми навыками являются Python, SQL, Postgre.

К сожалению это слишком поверхностный анализ, например обычно в требованиях указывают SQL, как замена всем типам SQL (MySQl, Postgre, MSSQL, Oracle и т.д). Реже добавляют к основному SQL навыку более точные требования к SQL или указывают четко опреденный SQL

Но в общем охарактеризовать ситуацию мы можем, что знания Python, SQL являются ключевыми, и претендентам обладающим этими навыками легче будет найти работу

Также, что интересно, среднее кол-во навыков лежит в диапазоне от 6 до 7 навыков, т.е необходимо сделать более глубокую сегментацию, чтобы сочетания каких навыков более восстребованы.

И характерен достаточно резкий рост зарплаты согласно Опыту работы

# Общий вывод по проекту

Отметим, что в рамках данного проекта не прооизводилась проверка на выбросы, поэтому данные могут быть немного искажены.

Также тяжело сказать, насколько профессия DS востребована относительно других профессий, так как мы выделили более менее четко сегмент DS специальностей, а остальные просто скинули в одну кучу, и понятия не имеем сколько уникальных специальностей в этой куче.

Что мы четко можем видеть, средняя зарплата DS специалиста в зависимости от опыта лежит в диапазоне от 75к до 245К рублей.
Также нет ни одного требования к опыту более 6 лет, что может свидетельствовать о том, что при большем опыте зарплата может также существенно расти.

HH.RU работает не только с регионом Россия, но также и с другими странами.

Некорректный момент, что таблица areas содержит как страны, так и города, из-за чего сегментирование ни по городам, ни по странам не является корректным.

Также нет четкой категоризации по вакансиям и навыкам.

Учитывая все вышесказанное, все наши выводы не имеют особой ценности, пока данные не будут приведены к более "чистому" виду, а именно:
* Выявление категорий для вакансий
* Выявление категорий для навыков
* Ведение дополнительного признака в таблице areas - Страна. Вопрос что делать с городом, или уточнять, или в случае небольших стран и возможности удаленной работы и т.д., приводить к одному городу...В вообщем нужен более глубокий анализ тут тоже





In [46]:
# Напишем запрос, который вернет нам среднюю зарплату в зависимости от опыта
# для DS вакансий и Других вакансий

# Создаем условие отбора НЕ относящихся к DS вакансиям
where_DS_NOT = '''    
      (
        v.name !~~* any (array[
          '%%data%%scientist%%',
          '%%data%%science%%',
          '%%исследователь%%данных%%',
          '%%machine%%learning%%',
          '%%машинн%%обучен%%'      
        ])
      or (
          v.name ~~* '%%HTML%%'
          and v.name !~~ '%%ML%%'
        )
      )
'''

# используем структуру запроса WITH, для более удобного чтения и понимания.
# Из уже выбранного множества только DS вакансии,высчитываем среднее количество навыков
sql_dop_1 = f'''

    (
    WITH DSOnly AS (
    SELECT
      *
    FROM public.vacancies as v
    WHERE
    {where_DS_NOT}
    )
    
    SELECT
      'Другие вакансии' as vacance_type
      ,ds.experience
      --округляем до 2 знаков после запятой
      ,ROUND(
          --высчитываем среднее по всем записям
          AVG(
            -- так как по условию задачи необходимо посчитать среднюю зарплату, при условии, что зарплата указана 
            -- как и минимальная так и максимальная для записи, или же в случае если уцказана только одна из эарплат, взять ее
            -- применяем функцию coalesce, где входными параметрами являются приоритеты расчета зарплат указанные по их приоритету
            coalesce((ds.salary_from+ds.salary_to)/2, ds.salary_from, ds.salary_to) 
        ),0) as average_salary_DS
    FROM DSOnly as ds 
    GROUP BY ds.experience  
    ORDER BY average_salary_ds desc NULLS LAST
    )

    UNION

    (
    WITH DSOnly AS (
    SELECT
      *
    FROM public.vacancies as v
    WHERE
    {where_DS_Only}
    )
    
    SELECT
      'DS вакансии' as vacance_type
      ,ds.experience
      --округляем до 2 знаков после запятой
      ,ROUND(
          --высчитываем среднее по всем записям
          AVG(
            -- так как по условию задачи необходимо посчитать среднюю зарплату, при условии, что зарплата указана 
            -- как и минимальная так и максимальная для записи, или же в случае если уцказана только одна из эарплат, взять ее
            -- применяем функцию coalesce, где входными параметрами являются приоритеты расчета зарплат указанные по их приоритету
            coalesce((ds.salary_from+ds.salary_to)/2, ds.salary_from, ds.salary_to) 
        ),0) as average_salary_DS
    FROM DSOnly as ds 
    GROUP BY ds.experience  
    ORDER BY average_salary_ds desc NULLS LAST
    )
    ;
  '''

In [47]:
data_ds_exp_salary =  sqlExecution(sql_dop_1)
data_ds_exp_salary

,vacance_type,experience,average_salary_ds
0,Другие вакансии,От 3 до 6 лет,133211.0
1,DS вакансии,От 3 до 6 лет,243115.0
2,DS вакансии,Более 6 лет,NaN
3,Другие вакансии,Более 6 лет,166256.0
4,DS вакансии,Нет опыта,74643.0
5,DS вакансии,От 1 года до 3 лет,139675.0
6,Другие вакансии,От 1 года до 3 лет,76541.0
7,Другие вакансии,Нет опыта,40472.0


In [48]:
# пишем простенькую функцию чтобы добавить сортировку по опыту для графика
def changeSort(experience):
    if experience =='Нет опыта':
        return 1
    elif experience == "От 1 года до 3 лет":
        return 2
    elif experience == "От 3 до 6 лет":
        return 3
    elif experience == "Более 6 лет":
        return 4

# Создаем колонку с данными для сортировки
data_ds_exp_salary['Sorting'] = data_ds_exp_salary.apply(lambda x: changeSort(x.experience), axis = 1)

# Получаем максимальное значение зарплаты, для дополнительного отображения разницы в процентах вмех колонок от максимума
max = data_ds_exp_salary.average_salary_ds.max() 

# Создаем колонку с данными процентов
data_ds_exp_salary['Percent'] = data_ds_exp_salary.average_salary_ds.apply(lambda x: round(x / max, 4))
# сортируем по новому признаку Sorting
data_ds_exp_salary.sort_values(by = ['Sorting'], inplace=True)

fig = px.bar(
    data_ds_exp_salary,
    x='experience',
    y='average_salary_ds',
    color='vacance_type', 
    color_discrete_sequence =['RGB(205,170,125)','RGB(100,149,237)'],
    hover_data = ["Percent","vacance_type"], #добавляем дополнительные поля для вывода в хинтах
    # text='vacance_type', # Дополнительно указываем в каждом Баре, к какой группе относится бар,
                  # в данном графике я счиатаю это добавляет информативности 
    
)

fig.update_traces(
    hoverlabel_namelength = 0, # убираем навязчивый левый надпись в ховере
    textposition='inside', # проценты пишем над баром
    texttemplate='<b style="color: #3C4048; font-size:11px;">   %{customdata[1]}</b>',
    hovertemplate="".join([
        "<b style='color: #393E46'>%{customdata[1]}</b>" # вытягиваем данные из переменной указаной в hover_data[]      
        "<br>Зарплата (руб): %{y}",
        "<br>",
        "Процент: %{customdata[0]:,.2%}", # вытягиваем данные из переменной указаной в hover_data[]
                                          # и форматируем чтобы проценты показывало корректно   
        
    ]),
)

fig.update_layout(
    barmode='group',
    hovermode="x unified",
    title="Зависимость средней заработной платы в рублях от опыта",
    font = dict(size=10), # уменьшим размер шрифта заголовка графика, а то не влезает
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100,
      l=100
    ),
    legend=dict(# играемся с параметрами легенды
        borderwidth = .5,# ширина рамки в полпикселя
        bordercolor = '#cbd7de',# цвет рамки
        bgcolor = '#f2faff',# цвет фона
        orientation="h", # показать сверху
        yanchor="bottom",# вертикальное выравнивание по нижней части легенды относительно холста
        y=1.01, # сдвигаем на 1% выше, чтобы не сливалась нижняя граница рамки легенды с графиком
        xanchor="right", # выравнивание блока легенды по правому краю
        x=1, # никаких сдвигов не надо, будет на уровне с графиком по правому краю
        title_text="", # убираем title для легенды, в данном случае понятно без лишних текстов
    ),      
    yaxis=dict(
        title_text="Средняя зарплата (руб)", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
        range = (0,30e4), # увеличивем диапазон отражения по оси ординат, 
                          # чтобы не срезалась отрисовка процентов над левым баром        
    ),
    xaxis=dict(
        title_text="Опыт", # меняем название заголовка оси абсцисс
        titlefont=dict(size=15),  # ее размер 
    ), 
  paper_bgcolor="#eaf7ff",# цвет фона холста
)

fig.show()


Поверхностное разделение по зависимости Опыта к зарплате между DS вакансиями и Другими, показывает, что в любых сегментах Опыта (кроме более 6 лет, там вообще нет данных по DS),
Зарплата DS специалиста почти в 2 раза выше, чем любой другой специальности.

Отсутствие Опыта более 6 лет, говорит, что специальность для HH.RU достаточно молодая на рынке труда

Также видна "крутая" зависимость роста зарплаты от опыта, что для DS, что для других специальностей.

Средняя зарплата DS вообще без опыта, составляет 74к, что является соразмерной средней зарплатой для других специальностей с опытом 1 - 3 года, что несколько странно. Это говорит что, вообще нет смысла устраиваться на работу по другой специальности и набирать опыт, когда сразу можно можно попытаться протиснуться без опыта на такие же денежные условия.

In [115]:
# Запрос показывающий какие навыки чаще всего присутствуют в списке навыков вакансий DS
sql_dop_2 = f'''
    
    WITH DSOnly as (
    SELECT
      *
    FROM public.vacancies as v
    WHERE
    {where_DS_Only}
    )
    
    SELECT
      -- разделим строку с ключевыми навыками по символу табуляции
      -- чтобы узнать какие навыки указывают в вакансиях для DS
      regexp_split_to_table(ds.key_skills, chr(9)) as skill
      ,count(*) as skill_count
      --посчитаем процент количества каждого из навыков
      ,round(count(*) * 100.0 / count(count(*)) over(),2) as pers
    FROM DSOnly as ds
    GROUP BY skill
    ORDER BY skill_count desc
    LIMIT 20
    ;
  '''

In [116]:
# Сохраним DataFrame, так как это будет иcпользовано в 4 дополнительном исследовании
topSkill = sqlExecution(sql_dop_2)
topSkill


,skill,skill_count,pers
0,Python,348,80.93
1,SQL,191,44.42
2,Machine Learning,114,26.51
3,Git,66,15.35
4,Математическая статистика,63,14.65
5,Data Analysis,54,12.56
6,Data Science,52,12.09
7,Pandas,52,12.09
8,ML,49,11.40
9,Английский язык,48,11.16


Как мы видим:
* В 81% одним из навыков в вакансии DS является Python
* На втором месте идет SQL с 45%
* Machine learning и ML вместе занимают около 38%
* Далее следует Анализ данных и Data Analysis около 25%

Т.е для профессии DS имея в списке навыков выше указанные навыки, шанс быстро найти работу очень высок


In [ ]:
# Не очень интересный анализ, но все же, получим списки Городов,
# где специалисты DS больше всего востребованы
sql_dop_3 = f'''
    WITH DSOnly as (
    SELECT
      *
    FROM public.vacancies as v
    WHERE
    {where_DS_Only}
    )
    
    SELECT 
      a.name,
      count(distinct ds.id) as vacancies_count
    FROM DSOnly ds
    INNER JOIN public.areas as a ON a.id = ds.area_id
    GROUP by a.name
    ORDER by vacancies_count desc
    --10 запись выпадает на Армению, не будем сравнивать ее с городами
    LIMIT 9
    ;
  '''

In [ ]:
# Скачаем список городов миллиоников, выберем только необходимые нам признаки
cities_info = pd.read_html(url)[1].iloc[:,1:3]

# Переименуем название признаков покороче
cities_info.columns = ['Город','Pop']

# Получим DataFrame с ТОП9 городв по количеству вакансий для DS
data_ds_areas = sqlExecution(sql_dop_3)

# Подтянем к нашей таблице, данные по населению из скачанной таблице
ci = pd.merge(data_ds_areas,cities_info, how='left', left_on='name',\
  right_on='Город')[['name','vacancies_count','Pop']]

# к сожалению не для всех городов есть информация, можно искать другие таблицы, но в нашем случае 
# нам проще вручную добить недостающие данные
ci.Pop.iloc[[5,6,7]] = [1777, 1975, 543]

# Уберем все пробелы и приведем к целому типу для дальнейших расчетов признак Pop
ci.Pop = ci.Pop.apply(lambda x : int(str(x).replace(' ','')))
# Высчитаем "виртуальный" признак который даст понять соотношение кол-ва вакансий к населению города
ci['Ratio'] = round(ci.apply(lambda x: (x.vacancies_count / x.Pop), axis=1)*1000,2)


In [ ]:
# Построим график в отдельной ячейке, чтобы не городит в одной куче
fig = px.line(ci, 
              x="name", 
              y="Ratio",
              hover_data=['vacancies_count','Pop'] 
              )

# Меняем стандартный hover на наш
fig.update_traces(
    mode="markers+lines",
    hovertemplate="".join([
        "<b style='color: yellow'>%{x}</b>" # вытягиваем данные из переменной указаной в hover_data[]      
        "<br>Соотношение: %{y}",
        "<br>Количество вакансий: %{customdata[0]}",
        "<br>Население (тыс): %{customdata[1]}",
    ]),
    hoverlabel_namelength = 0, # убираем вывод ховера сверху,
                               # потому как хочу убрать двоеточие в стандартном выводе и добавить стиль bold
)

fig.update_layout(
    title="Отношение количества вакансий к населению города",
    font = dict(size=10), # уменьшим размер шрифта заголовка графика, а то не влезает
    autosize=False,# скажи нет авторазмеру
    width=700,# фиксируем ширину в px
    height=500,# фиксируем высоту в px
    margin=dict(
      r=30, #задаем отступ в 30 px справа для всего холста
      b=60, # задаем отступ в 60 px снизу для всего холста
      t=100,
      l=100
    ),
    legend=dict(# играемся с параметрами легенды
        borderwidth = .5,# ширина рамки в полпикселя
        bordercolor = '#cbd7de',# цвет рамки
        bgcolor = '#f2faff',# цвет фона
        orientation="h", # показать сверху
        yanchor="bottom",# вертикальное выравнивание по нижней части легенды относительно холста
        y=1.01, # сдвигаем на 1% выше, чтобы не сливалась нижняя граница рамки легенды с графиком
        xanchor="right", # выравнивание блока легенды по правому краю
        x=1, # никаких сдвигов не надо, будет на уровне с графиком по правому краю
        title_text="", # убираем title для легенды, в данном случае понятно без лишних текстов
    ),     
    yaxis=dict(
        titlefont=dict(size=12),  # ее размер 
        title_text='Отношение',
        range = [0,20]
    ),
    xaxis=dict(
        title_text = ''
    ), 
  paper_bgcolor="#eaf7ff",# цвет фона холста
)


fig.show()

Добавив соотношение количества вакансий к населению, мы видим, что на территории России, восстребованность в специалистах несколько отличается.

* Лидерами являются Москва, нижний Новгород и Томск.
* Санкт-Петербург, Новосибирск, Казань менее заинтересованы в вакансии DS
* Что интересно, в Екатеринбурге интерес в 4 раза ниже чем например в Москве(ТОП) или в 3 раза с предпоследним городом России.
* Также в выборку, городов с самым большим предложением вакансий для DS попали Алматы (Казахстан)и Минск(Беларусь). Однако предложения по этим республикам не такое высокое. Скорее всего это может говорить, что HH.RU более Российский ресурс работодатели других стран больше выбирают местные HR агенства. Но это только предположение, чтобы точнее сказать, надо детальнее углубится в этот момент


In [ ]:
sql_dop_4 = f'''
    
    WITH DSOnly as (
    SELECT
      *
    FROM public.vacancies as v
    WHERE
    {where_DS_Only}
    )
    
    SELECT 
      ds.id
      -- разделим строку с ключевыми навыками по символу табуляции,
      -- чтобы узнать какие навыки указывают в вакансиях для DS 
      ,regexp_split_to_table(ds.key_skills, chr(9)) as skill
    FROM DSOnly ds
       ;
  '''



In [ ]:
# Тут выйдем за рамки только SQL
data_skill_df = sqlExecution(sql_dop_4)
# Делаем pivot по вакансии и навыку
data_skill_pivot = pd.\
  pivot_table(data_skill_df, index=['id'], columns= 'skill', \
  fill_value=0, aggfunc={'skill':'count'}).reset_index()
  
# превращаем в 1 DataFrame без лишних Axis

data_skill_pivot = pd.concat([data_skill_pivot.id,data_skill_pivot.skill],axis=1) 

# Из полученного DF во втором дополнительном исследовании выбираем ТОП5 
# навыков для DS специалистов и приводим их к типу list
tpSkill = list(topSkill.skill.head(5))

# напишем простенькую функцию, которая будет для входящей строки из DataFrame вычислять
# какие навыки указаны и проверять какие из них в ТОП5, если да, то добавлять в list на отправку
# Так мы получим представление о популярном наборе навыков для DS специалиста
def make_list(row):
  tp = row.to_numpy().nonzero()[0]
  return sorted(list(
    (
        set(row[tp[1:]].keys()))
      & set(tpSkill)
    )              
  )
# Вводим новый признак, где находим только навыки из ТОП20 навыков(отсортированные)
data_skill_pivot['topSkills'] = data_skill_pivot.apply(make_list, axis = 1)
# Выводим Сочетания навыков
finalTable = pd.concat([data_skill_pivot.topSkills.value_counts(),\
        round(data_skill_pivot.topSkills.value_counts(normalize=True)*100,2)],\
        axis=1).\
        reset_index().\
        rename(columns={'index':'set','topSkills':'quantity','topSkills':'%'})
finalTable.columns = ['set','quantity','%']
finalTable

,set,quantity,%
0,[Python],91,21.02
1,"[Python, SQL]",63,14.55
2,"[Machine Learning, Python]",56,12.93
3,[],40,9.24
4,"[Python, SQL, Математическая статистика]",31,7.16
5,"[Machine Learning, Python, SQL]",29,6.70
6,"[Git, Python, SQL]",27,6.24
7,[SQL],24,5.54
8,"[Git, Python]",17,3.93
9,[Git],11,2.54


Данным исследованием мы узнали наиболее частые сочетания требуемых навыков для ТОП5 встречаемых навыков.
Тут правильно было бы некоторые навыки категоризировать, (но уже често надоело ) и думаю для текущего проекта хватит), имею ввиду такие навыки, как ML и Machine Learning и т.д., что считается одним навыком.

Самым популярным является сочетание Python с навыками не попавшими в TOP5 и составляет 21%
* Вторым по популярности идет Python вместе c SQL - 14.5%
* Третьим по популярности идет Python вместе c Machine Learning - 14.5%
* На 4-ом месте вакансии, которые не требуют ни одного из TOP5 навыков
* На 6 месте Machine Learning, Python, SQL с 6.7 %

В принципе, данный расклад доказывает выводы сделаные в дополнительных условиях под номером 2
и делает это несколько в ином разрезе, где мы уже четче видит основные сочетания навыков, необходмые в большинстве вакансий.

Учитывая что работодатель указывает не все требования, можем считать что самым распространенным требованием является сочетание
* SQL, Python, ML, Мат Статистика, Git

Людям с такими навыками будет проще найти работу по специальности DS

Также было бы интересно изучить 4-е место, где нет ни одного навыка из ТОП5 навыков. Возможно это требования более специфических требований и возможно более денежные вакансии.


